## Predict Hurling Pass Probabilities

In [3]:
import pandas as pd
import cv2

df = pd.read_csv("Hurling_intersquad_passes_final_data.pipe", sep="|")

In [4]:
df.head(1)

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,haversine_smooth,mph,mph_smoothed,haversine_smooth_10,mph_smoothed_10,ft_to_ball,ft_traveled,pixels_out,x_out,y_out
0,3,1510.0,449.5,Eric,H,[ 41.70953108 -72.58555477],41.709531,-72.585555,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,63.555697,0.0,[850.20866903 435.28057347],850.208669,435.280573


## Define Pass Completion
The intention is to help make good pass decisions:<br>
Therefore, a pass will be considered successful if the ball arrives to a player on the same team, without a member of the opposing team defending it. The receiver of the pass doesn't need to acquire possession because that depends more on the skill of the player than on the decision to pass. For example, a pass will be considered successful even if the receiver drops it.

## Identify Passes and Explore Outcomes

In [5]:
ball_actions = {3:"player12",
                20:"Pass",
                73:"player6",
                76:"Lost",
                 129: "Eric",
                 132:"Pass",
                 153:"player5",
                189:"Pass",
                268:"player15",
                277:"Lost",
                435:"player18",
                542:"Pass",
                650:"player10",
                731:"Pass",
                818:"player13",
                819:"Lost",
                859:"player17",
                910:"Pass",
                933:"player11",
                986:"Pass",
                1040:"player5",
                1041:"Lost",
                1173:"player8",
                1227:"Pass",
                1258:"player10",
                1350:"Pass",
                1403:"player13",
                1448:"Pass",
                1492:"Eric",
                1494:"Lost",
                1547:"Eric",
                1617:"Pass",
                1740:"player14",
                1741:"Lost",
                1781:"player4",
                1782:"Pass",
                1885:"player7",
                1919:"Pass",
                1943:"player20",
                }

In [6]:
passes = [{"frame":20,
                 "passer":"player12",
                 "receiver": "player6",
                 "Outcome":True
                },
                {
                "frame":132,
                 "passer":"Eric",
                 "receiver": "player5",
                 "Outcome":True
                 },
                 {
                 "frame":189,
                 "passer":"player5",
                 "receiver": "player16",
                 "Outcome":False
                 },
                {
                 "frame":542,
                 "passer":"player18",
                 "receiver": "player10",
                 "Outcome":True
                 },
                 {
                 "frame":731,
                 "passer":"player10",
                 "receiver": "player13",
                 "Outcome":True
                 },
                 {
                 "frame":910,
                 "passer":"player17",
                 "receiver": "player11",
                 "Outcome":True
                 },
                 {
                 "frame":986,
                 "passer":"player11",
                 "receiver": "player5",
                 "Outcome":True
                 },
                 {
                 "frame":1227,
                 "passer":"player8",
                 "receiver": "player10",
                 "Outcome":True
                 },
                 {
                 "frame":1350,
                 "passer":"player10",
                 "receiver": "player13",
                 "Outcome":True
                 },
                 {
                 "frame":1448,
                 "passer":"player13",
                 "receiver": "player10",
                 "Outcome":False
                 },
                 {
                 "frame":1617,
                 "passer":"Eric",
                 "receiver": "player4",
                 "Outcome":False
                 },
                 {
                 "frame":1782,
                 "passer":"player4",
                 "receiver": "player7",
                 "Outcome":True
                 },
                 {
                 "frame":1919,
                 "passer":"player7",
                 "receiver": "player20",
                 "Outcome":True
                 }]
passes = pd.DataFrame(passes)

In [7]:
passes

,frame,passer,receiver,Outcome
0,20,player12,player6,True
1,132,Eric,player5,True
2,189,player5,player16,False
3,542,player18,player10,True
4,731,player10,player13,True
5,910,player17,player11,True
6,986,player11,player5,True
7,1227,player8,player10,True
8,1350,player10,player13,True
9,1448,player13,player10,False


In [8]:
passes['Outcome'].value_counts()

True     10
False     3
Name: Outcome, dtype: int64

In [9]:
print("Sample Size:", passes.shape[0])

Sample Size: 13


In [10]:
players = df[['Name', "Team"]].drop_duplicates().sort_values("Name").reset_index(drop=True)
players = players[players['Team'].isin(["H","F"])].reset_index(drop=True)
print("Players per Team on Screen")
players['Team'].value_counts()

Players per Team on Screen


H    11
F    11
Name: Team, dtype: int64

Obviously the sample size is tiny, so the model won't be very useful. However, we'll still create a demo model

## Determine Predictors

From game experience, I'm guessing the most important predictors would be: openness, distance, and speed
<br>
Distance and speed are obvious to calculate. However, openness can mean many things... Let's calculate a few basic fields: 
<ul>
<li>
distance to nearest defender. </li>
<li>
Closest defender speed divided by receiver speed. 
</li>
<li>
Is a defender between the passer and receiver?
</li>
<ul>

In [11]:
import haversine as hs
from haversine import Unit
import numpy as np
import math
 
def getAngle(a, b, c):
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

# Identify Defenders and Determine how many are in the path of the ball
def orthoProjection(ax, ay, bx, by, cx, cy):
    abx = bx - ax
    aby = by - ay
    acx = cx - ax
    acy = cy - ay
    t = (abx * acx + aby * acy) / (abx * abx + aby * aby)
    px = ax + t * abx
    py = ay + t * aby
    return px, py


In [12]:
ball_lost = [i for i in ball_actions.keys() if ball_actions[i] == "Lost"]

In [13]:
ball_lost

[76, 277, 819, 1041, 1494, 1741]

In [14]:

# I realize this code is not the most efficient, but it's not real-time so that's ok :)
# Also, the intersection and angle might not be precise because the formula's are for 2D space while longitude/latitude is spherical
# Determining openness is just an estimate, so we don't need exact precision
ball_carrier, ball_team = None, None
distance_df = []
for Frame, group in df.groupby("frame"):
    print(Frame)
    if Frame in ball_lost: # Don't do calculations if ball has been lost
        ball_carrier, ball_team = None, None
    teammates = []
    group = group[~group['lon'].isna()].copy()
    for index in group.index:
        if group.loc[index]['Team'] in ["H","F"]:
            if (list(group.loc[group['Name'] == 'ball'].to_dict()['lat'].values())[0], list(group.loc[group['Name'] == 'ball'].to_dict()['lon'].values())[0]) == (group.loc[index]['lat'], group.loc[index]['lon']):
                ball_carrier = group.loc[index]['Name']
                ball_team = group.loc[index]['Team']
    if ball_carrier and ball_team:
        group['nbr_of_interceptors'] = np.nan
        group['nearest_defender_dist'] = np.nan
        for index in group.index:
            subgroup = []
            # print(group.loc[index]['Name'], group.loc[index]['Team'])
            if group.loc[index]['Team'] == ball_team and group.loc[index]['Name'] != ball_carrier:
                if group.loc[index]['Team'] == "H":
                    subgroup = group[group['Team'] == "F"]
                elif group.loc[index]['Team'] == "F":
                    subgroup = group[group['Team'] == "H"]
                if type(subgroup) != list:
                    #relevant defenders
                    subgroup = subgroup[(~subgroup['lon'].isna()) & (~subgroup['lat'].isna())].copy()
                    d = {}
                    for i in subgroup.index:
                        # print("-", subgroup.loc[i]['Name'])
                        if (list(group.loc[group['Name'] == 'ball'].to_dict()['lat'].values())[0], list(group.loc[group['Name'] == 'ball'].to_dict()['lon'].values())[0]) != (subgroup.loc[i]['lat'], subgroup.loc[i]['lon']): # Player does not have ball
                            d[subgroup.loc[i]['Name']] = {}
                            d[subgroup.loc[i]['Name']]['distance'] = hs.haversine((group.loc[index]['lat'], group.loc[index]['lon']), (subgroup.loc[i]['lat'], subgroup.loc[i]['lon']), unit=Unit.MILES) * 5280
                            d[subgroup.loc[i]['Name']]['ball_coordinates'] = (list(group.loc[group['Name'] == 'ball'].to_dict()['lat'].values())[0], list(group.loc[group['Name'] == 'ball'].to_dict()['lon'].values())[0])
                            d[subgroup.loc[i]['Name']]['coordinates'] = (subgroup.loc[i]['lat'], subgroup.loc[i]['lon'])
                            d[subgroup.loc[i]['Name']]['receiver_coordinates'] = (group.loc[index]['lat'], group.loc[index]['lon'])
                            d[subgroup.loc[i]['Name']]['intercept_coordinates'] = orthoProjection(d[subgroup.loc[i]['Name']]['ball_coordinates'][0], d[subgroup.loc[i]['Name']]['ball_coordinates'][1], d[subgroup.loc[i]['Name']]['receiver_coordinates'][0], d[subgroup.loc[i]['Name']]['receiver_coordinates'][1], d[subgroup.loc[i]['Name']]['coordinates'][0], d[subgroup.loc[i]['Name']]['coordinates'][1])
                            d[subgroup.loc[i]['Name']]['sprint_distance'] = hs.haversine(d[subgroup.loc[i]['Name']]['coordinates'], d[subgroup.loc[i]['Name']]['intercept_coordinates'], unit=Unit.MILES) * 5280
                            d[subgroup.loc[i]['Name']]['ball_distance'] = hs.haversine(d[subgroup.loc[i]['Name']]['ball_coordinates'], d[subgroup.loc[i]['Name']]['intercept_coordinates'], unit=Unit.MILES) * 5280
                            d[subgroup.loc[i]['Name']]['overtake_distance'] = hs.haversine(d[subgroup.loc[i]['Name']]['ball_coordinates'], d[subgroup.loc[i]['Name']]['intercept_coordinates'], unit=Unit.MILES) * 5280
                            d[subgroup.loc[i]['Name']]['pass_distance'] = hs.haversine(d[subgroup.loc[i]['Name']]['receiver_coordinates'], d[subgroup.loc[i]['Name']]['ball_coordinates'], unit=Unit.MILES) * 5280
                            d[subgroup.loc[i]['Name']]['angle'] = getAngle(d[subgroup.loc[i]['Name']]['ball_coordinates'], d[subgroup.loc[i]['Name']]['receiver_coordinates'], d[subgroup.loc[i]['Name']]['coordinates'])
                            d[subgroup.loc[i]['Name']]['mph_smoothed_10'] = subgroup.loc[i]['mph_smoothed_10']
                            if d[subgroup.loc[i]['Name']]['sprint_distance'] / d[subgroup.loc[i]['Name']]['ball_distance'] < .5 and d[subgroup.loc[i]['Name']]['ball_distance'] <= d[subgroup.loc[i]['Name']]['pass_distance']: # Assumes a player can run half as fast at the ball (20mph vs 40 mph)
                                d[subgroup.loc[i]['Name']]['within_intercept_range'] = True
                            else:
                                d[subgroup.loc[i]['Name']]['within_intercept_range'] = False
                    # print(sum(d[name]['within_intercept_range'] for name in d.keys()))   
                    if group.loc[index]['Team'] == ball_team:
                        group.loc[index, "pass_distance"] = hs.haversine((group.loc[index]['lat'], group.loc[index]['lon']), (list(group.loc[group['Name'] == 'ball'].to_dict()['lat'].values())[0], list(group.loc[group['Name'] == 'ball'].to_dict()['lon'].values())[0]), unit=Unit.MILES) * 5280
                        group.loc[index, 'nbr_of_interceptors'] = sum(d[name]['within_intercept_range'] for name in d.keys())
                        group.loc[index, 'nearest_defender_dist'] = min(d[name]['distance'] for name in d.keys())
                        group.loc[index, "nearest_defender_angle"] = [d[name]['angle'] for name in d.keys() if d[name]['distance'] == min(d[name]['distance'] for name in d.keys())][0]
                        group.loc[index, "nearest_defender_speed"] = [d[name]['mph_smoothed_10'] for name in d.keys() if d[name]['distance'] == min(d[name]['distance'] for name in d.keys())][0]
                        group.loc[index, "nearest_defender_speed_rel"] = group.loc[index, "nearest_defender_speed"] / group.loc[index, "mph_smoothed_10"]
                        group.loc[index, "receiver_speed"] = group.loc[index, "mph_smoothed_10"]
                    # print(d)
                    teammates.append({group.loc[index]['Name']:d})
        
    distance_df.append(group)
distance_df = pd.concat(distance_df)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


/Users/ericstassen/opt/anaconda3/envs/vision37/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in double_scalars


1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961


df = distance_df['distances_mile'].apply(pd.Series)

## Prepare Training Dataset

In [15]:
modeling_df = pd.merge(passes,distance_df[["frame", "Name", "pass_distance", "nbr_of_interceptors", "nearest_defender_dist", "nearest_defender_angle", "nearest_defender_speed", "nearest_defender_speed_rel", "receiver_speed"]], how="left", left_on=['frame', 'receiver'], right_on=['frame', 'Name'])

In [16]:
modeling_df

,frame,passer,receiver,Outcome,Name,pass_distance,nbr_of_interceptors,nearest_defender_dist,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
0,20,player12,player6,True,player6,83.013798,1.0,12.178738,318.550939,8.901124,0.795583,11.188173
1,132,Eric,player5,True,player5,9.692406,0.0,15.176655,42.959883,9.242666,4.370869,2.114606
2,189,player5,player16,False,player16,50.707103,0.0,27.934074,270.382145,6.426632,0.676624,9.498079
3,542,player18,player10,True,player10,60.001782,2.0,50.430330,248.828058,2.942543,0.556849,5.284272
4,731,player10,player13,True,player13,104.023497,2.0,6.287208,212.476577,1.467505,0.782977,1.874263
5,910,player17,player11,True,player11,12.987240,0.0,30.683362,8.028550,5.349269,1.259303,4.247800
6,986,player11,player5,True,player5,48.472351,1.0,23.017667,143.019762,3.997890,0.259147,15.427091
7,1227,player8,player10,True,player10,15.811230,1.0,7.162794,38.317057,6.717765,1.013309,6.629531
8,1350,player10,player13,True,player13,18.168620,0.0,12.948788,349.955343,7.002072,1.272270,5.503604
9,1448,player13,player10,False,player10,25.580474,1.0,13.607012,1.089764,10.310118,1.363177,7.563302


## Train Model

In [17]:
modeling_df.drop(columns=['frame', 'passer', 'receiver', 'Name'], inplace=True)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(modeling_df.drop('Outcome',axis=1), 
                                                    modeling_df['Outcome'], test_size=0.10, 
                                                    random_state=100)

In [19]:
X_train

,pass_distance,nbr_of_interceptors,nearest_defender_dist,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
1,9.692406,0.0,15.176655,42.959883,9.242666,4.370869,2.114606
5,12.987240,0.0,30.683362,8.028550,5.349269,1.259303,4.247800
4,104.023497,2.0,6.287208,212.476577,1.467505,0.782977,1.874263
2,50.707103,0.0,27.934074,270.382145,6.426632,0.676624,9.498079
0,83.013798,1.0,12.178738,318.550939,8.901124,0.795583,11.188173
11,131.602801,0.0,2.485127,118.208366,3.173192,0.974446,3.256406
9,25.580474,1.0,13.607012,1.089764,10.310118,1.363177,7.563302
7,15.811230,1.0,7.162794,38.317057,6.717765,1.013309,6.629531
3,60.001782,2.0,50.430330,248.828058,2.942543,0.556849,5.284272
12,30.999553,1.0,16.117824,176.741873,5.942840,0.616905,9.633307


In [20]:
y_train

1      True
5      True
4      True
2     False
0      True
11     True
9     False
7      True
3      True
12     True
8      True
Name: Outcome, dtype: bool

In [21]:
X_test

,pass_distance,nbr_of_interceptors,nearest_defender_dist,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
10,67.565699,3.0,32.206218,264.231690,3.34806,0.741712,4.513959
6,48.472351,1.0,23.017667,143.019762,3.99789,0.259147,15.427091


In [22]:
y_test

10    False
6      True
Name: Outcome, dtype: bool

In [23]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [24]:
list(predictions), list(y_test)

([True, False], [False, True])

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00       1.0
        True       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



In [26]:
# Training Data is probably too small...
# Let's artificially expand the dataframe
ball_actions_list = list(ball_actions.keys())
passes_expanded = []
for frame in passes['frame'].to_list():
    for i in range(frame, ball_actions_list[ball_actions_list.index(frame)+1]):
        frame_record = passes.loc[passes['frame'] == frame].copy()
        frame_record['frame'] = i
        passes_expanded.append(frame_record)
passes_expanded = pd.concat(passes_expanded).reset_index(drop=True)

In [27]:
passes_expanded

,frame,passer,receiver,Outcome
0,20,player12,player6,True
1,21,player12,player6,True
2,22,player12,player6,True
3,23,player12,player6,True
4,24,player12,player6,True
...,...,...,...,...
798,1938,player7,player20,True
799,1939,player7,player20,True
800,1940,player7,player20,True
801,1941,player7,player20,True


In [28]:
modeling_df = pd.merge(passes_expanded,distance_df[["frame", "Name", "pass_distance", "nbr_of_interceptors", "nearest_defender_dist", "nearest_defender_angle", "nearest_defender_speed", "nearest_defender_speed_rel", "receiver_speed"]], how="left", left_on=['frame', 'receiver'], right_on=['frame', 'Name'])

In [29]:
modeling_df

,frame,passer,receiver,Outcome,Name,pass_distance,nbr_of_interceptors,nearest_defender_dist,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
0,20,player12,player6,True,player6,83.013798,1.0,12.178738,318.550939,8.901124,0.795583,11.188173
1,21,player12,player6,True,player6,80.669006,1.0,12.394691,317.925405,9.342192,0.818778,11.409916
2,22,player12,player6,True,player6,78.509331,1.0,12.619397,317.242427,9.490431,0.835308,11.361601
3,23,player12,player6,True,player6,76.573348,1.0,12.434292,317.522120,9.415295,0.844216,11.152714
4,24,player12,player6,True,player6,74.878263,1.0,11.922606,318.587542,10.324335,0.934065,11.053128
...,...,...,...,...,...,...,...,...,...,...,...,...
798,1938,player7,player20,True,player20,5.305106,0.0,15.491738,174.839185,5.759519,0.722824,7.968082
799,1939,player7,player20,True,player20,4.057866,0.0,15.365408,179.212335,6.181229,0.784242,7.881786
800,1940,player7,player20,True,player20,2.411682,0.0,15.662977,187.654894,6.452204,0.769754,8.382162
801,1941,player7,player20,True,player20,2.121363,0.0,14.394957,177.557529,7.035955,0.729065,9.650650


In [30]:
modeling_df.drop(columns=['frame', 'passer', 'receiver', 'Name'], inplace=True)

In [31]:
modeling_df[modeling_df.isna().any(axis=1)]

,Outcome,pass_distance,nbr_of_interceptors,nearest_defender_dist,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
110,False,31.953929,1.0,7.170166,177.983829,NaN,NaN,3.072385
111,False,31.329647,1.0,7.492435,180.814082,NaN,NaN,3.212635
112,False,30.695252,1.0,7.973706,183.170966,NaN,NaN,3.276838
113,False,30.005438,2.0,8.701615,186.229701,NaN,NaN,3.166892
114,False,29.465260,2.0,9.274662,188.054467,NaN,NaN,3.400278
115,False,28.890470,2.0,8.815982,187.089728,NaN,NaN,3.397728
116,False,28.431361,1.0,9.054436,187.401665,NaN,NaN,4.168231
117,False,27.963911,1.0,8.682004,185.803445,NaN,NaN,5.185428
118,False,27.415503,1.0,7.879476,184.130302,NaN,NaN,5.448917
119,False,26.635367,2.0,7.966070,184.527864,NaN,NaN,5.630567


In [32]:
passes_expanded.drop(index=list(modeling_df[modeling_df.isna().any(axis=1)].index), inplace=True)

In [33]:
modeling_df.dropna(inplace=True)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(modeling_df.drop('Outcome',axis=1), 
                                                    modeling_df['Outcome'], test_size=0.10, 
                                                    random_state=100)

In [35]:
X_train

,pass_distance,nbr_of_interceptors,nearest_defender_dist,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
427,15.843629,1.0,9.114526,29.098899,6.864626,0.821907,8.352075
510,25.244722,1.0,13.701207,1.834837,9.031401,1.163866,7.759832
539,11.795943,2.0,6.180294,11.348283,3.201444,0.980380,3.265511
612,9.703123,0.0,16.606242,299.766749,9.723571,1.297095,7.496424
159,58.165495,2.0,50.785161,248.819571,2.183747,0.388095,5.626831
...,...,...,...,...,...,...,...
588,28.409986,1.0,23.026432,277.149081,6.274606,1.290074,4.863755
53,9.692406,0.0,15.176655,42.959883,9.242666,4.370869,2.114606
360,6.779181,0.0,26.644833,359.020412,8.139832,3.702593,2.198414
79,48.324300,0.0,26.368809,259.423130,6.339187,0.747629,8.479058


In [36]:
y_train.value_counts()

True     500
False    213
Name: Outcome, dtype: int64

In [37]:
# from sklearn.linear_model import LogisticRegression

# logmodel = LogisticRegression()
# logmodel.fit(X_train,y_train)
# predictions = logmodel.predict(X_test)
# predictions_proba = logmodel.predict_proba(X_test)

import statsmodels.api as sm
logmodel = sm.Logit(y_train, X_train).fit()
print(logmodel.summary())
predictions = (logmodel.predict(X_test) > .5).astype(int)
predictions_proba = logmodel.predict(X_test)

Optimization terminated successfully.
         Current function value: 0.466499
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                Outcome   No. Observations:                  713
Model:                          Logit   Df Residuals:                      706
Method:                           MLE   Df Model:                            6
Date:                Sat, 23 Sep 2023   Pseudo R-squ.:                  0.2350
Time:                        22:17:54   Log-Likelihood:                -332.61
converged:                       True   LL-Null:                       -434.78
Covariance Type:            nonrobust   LLR p-value:                 2.267e-41
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
pass_distance                  0.0382      0.006      6.352      0.000      

In [38]:
predictions

464    1
227    1
688    1
500    1
401    1
      ..
384    1
143    0
602    1
552    0
720    1
Length: 80, dtype: int64

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))
# Little Better: 86% Accuracy

              precision    recall  f1-score   support

       False       0.88      0.61      0.72        23
        True       0.86      0.96      0.91        57

    accuracy                           0.86        80
   macro avg       0.87      0.79      0.81        80
weighted avg       0.86      0.86      0.85        80



In [40]:
predictions_proba

464    0.782227
227    0.804069
688    0.996116
500    0.778953
401    0.786440
         ...   
384    0.699797
143    0.149296
602    0.813386
552    0.025258
720    0.897880
Length: 80, dtype: float64

## Predict Probability of Pass throughout video

In [41]:
distance_df

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,pixels_out,x_out,y_out,nbr_of_interceptors,nearest_defender_dist,pass_distance,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
0,3,1510.0,449.5,Eric,H,[ 41.70953108 -72.58555477],41.709531,-72.585555,NaN,NaN,...,[850.20866903 435.28057347],850.208669,435.280573,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,3806.5,583.5,ball,N,[ 41.70895713 -72.58558384],41.708957,-72.585584,NaN,NaN,...,[1588.24500435 615.79902289],1588.245004,615.799023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1542.5,390.5,player8,F,[ 41.70954044 -72.58551292],41.709540,-72.585513,NaN,NaN,...,[850.05148978 377.45404349],850.051490,377.454043,2.0,15.302462,68.741884,289.537910,NaN,NaN,NaN
7,3,880.5,494.5,player3,H,[ 41.70963465 -72.58559718],41.709635,-72.585597,NaN,NaN,...,[636.54562365 455.6150823 ],636.545624,455.615082,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,3373.5,919.5,player7,H,[ 41.70911253 -72.58577236],41.709113,-72.585772,NaN,NaN,...,[1393.88812443 800.41396017],1393.888124,800.413960,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45632,1961,533.5,430.0,player13,F,[ 41.70971542 -72.5855676 ],41.709715,-72.585568,41.709716,-72.585568,...,[479.9470292 380.02873703],479.947029,380.028737,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45633,1961,1977.5,553.0,Referee,R,[ 41.70941444 -72.58560814],41.709414,-72.585608,41.709415,-72.585608,...,[1023.72638689 540.98795873],1023.726387,540.987959,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45634,1961,3149.0,801.0,player7,H,[ 41.70915327 -72.58572214],41.709153,-72.585722,41.709153,-72.585721,...,[1354.4531825 735.26040255],1354.453183,735.260403,1.0,1.442488,12.569617,12.989707,8.776878,0.881311,9.958884
45635,1961,3207.5,800.0,player1,F,[ 41.70914046 -72.58572117],41.709140,-72.585721,41.709143,-72.585721,...,[1370.13416849 735.96871849],1370.134168,735.968718,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
distance_df[(distance_df["pass_distance"] == np.inf) | (distance_df["nbr_of_interceptors"] == np.inf) | (distance_df["nearest_defender_dist"] == np.inf) | (distance_df["nearest_defender_angle"]  == np.inf) | (distance_df["nearest_defender_speed"] == np.inf) | (distance_df["nearest_defender_speed_rel"] == np.inf) | (distance_df["receiver_speed"] == np.inf)][["pass_distance", "nbr_of_interceptors", "nearest_defender_dist", "nearest_defender_angle", "nearest_defender_speed", "nearest_defender_speed_rel", "receiver_speed"]]

,pass_distance,nbr_of_interceptors,nearest_defender_dist,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
43879,58.478039,1.0,16.253251,330.373199,1.591911,inf,0.0
43907,58.987846,1.0,16.328397,329.841001,1.578623,inf,0.0
43936,59.552135,1.0,16.337426,329.480567,1.546710,inf,0.0
43948,60.172042,2.0,16.344239,329.091383,1.498588,inf,0.0


In [43]:
distance_df[distance_df['frame'] == 700]

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,pixels_out,x_out,y_out,nbr_of_interceptors,nearest_defender_dist,pass_distance,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
16105,700,3033.5,398.0,player16,H,[ 41.70916335 -72.58545945],41.709163,-72.585459,41.709162,-72.585460,...,[1391.4521156 439.93783345],1391.452116,439.937833,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16106,700,2603.0,508.0,player6,F,[ 41.70927924 -72.58556387],41.709279,-72.585564,41.709278,-72.585564,...,[1229.09124932 523.49049646],1229.091249,523.490496,1.0,26.820124,59.631016,302.948236,3.248151,0.857695,3.787071
16107,700,3393.5,1008.0,player20,H,[ 41.70911444 -72.58580605],41.709114,-72.585806,41.709114,-72.585806,...,[1385.30454361 840.91752267],1385.304544,840.917523,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16108,700,2911.5,596.5,Referee,R,[ 41.70920193 -72.58561477],41.709202,-72.585615,41.709201,-72.585615,...,[1316.39268072 600.80230066],1316.392681,600.802301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16109,700,918.5,761.5,player11,H,[ 41.70955924 -72.58572249],41.709559,-72.585722,41.709559,-72.585722,...,[727.70834489 660.8110095 ],727.708345,660.811009,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16110,700,3179.5,483.5,player15,F,[ 41.70912617 -72.58552711],41.709126,-72.585527,41.709126,-72.585527,...,[1421.43014909 521.09439794],1421.430149,521.094398,1.0,12.566444,45.883506,229.313328,3.248151,1.279630,2.538352
16111,700,2646.5,374.0,player5,H,[ 41.709276 -72.58545569],41.709276,-72.585456,41.709275,-72.585456,...,[1257.87207407 402.26349593],1257.872074,402.263496,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16112,700,2119.0,661.0,player4,H,[ 41.70937115 -72.58566518],41.709371,-72.585665,41.709371,-72.585665,...,[1072.56793259 624.89394538],1072.567933,624.893945,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16113,700,130.0,577.5,player13,F,[ 41.70972506 -72.5856521 ],41.709725,-72.585652,41.709725,-72.585652,...,[407.91523537 504.17915157],407.915235,504.179152,2.0,5.174211,108.237503,219.630037,0.989102,1.329230,0.744116
16114,700,2449.5,1079.5,player9,F,[ 41.70928206 -72.58582909],41.709282,-72.585829,41.709282,-72.585830,...,[1157.97845397 855.08456857],1157.978454,855.084569,9.0,3.391401,155.887840,89.849015,2.624520,0.947142,2.770989


In [44]:
distance_df.loc[(~distance_df["pass_distance"].isna()) & (~distance_df["nbr_of_interceptors"].isna()) & (~distance_df["nearest_defender_dist"].isna()) & (~distance_df["nearest_defender_angle"].isna()) & (~distance_df["nearest_defender_speed"].isna()) & (~distance_df["nearest_defender_speed_rel"].isna()) & (~distance_df["receiver_speed"].isna())].loc[distance_df['frame'] == 700]

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,pixels_out,x_out,y_out,nbr_of_interceptors,nearest_defender_dist,pass_distance,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed
16106,700,2603.0,508.0,player6,F,[ 41.70927924 -72.58556387],41.709279,-72.585564,41.709278,-72.585564,...,[1229.09124932 523.49049646],1229.091249,523.490496,1.0,26.820124,59.631016,302.948236,3.248151,0.857695,3.787071
16110,700,3179.5,483.5,player15,F,[ 41.70912617 -72.58552711],41.709126,-72.585527,41.709126,-72.585527,...,[1421.43014909 521.09439794],1421.430149,521.094398,1.0,12.566444,45.883506,229.313328,3.248151,1.279630,2.538352
16113,700,130.0,577.5,player13,F,[ 41.70972506 -72.5856521 ],41.709725,-72.585652,41.709725,-72.585652,...,[407.91523537 504.17915157],407.915235,504.179152,2.0,5.174211,108.237503,219.630037,0.989102,1.329230,0.744116
16114,700,2449.5,1079.5,player9,F,[ 41.70928206 -72.58582909],41.709282,-72.585829,41.709282,-72.585830,...,[1157.97845397 855.08456857],1157.978454,855.084569,9.0,3.391401,155.887840,89.849015,2.624520,0.947142,2.770989
16115,700,3065.0,432.5,player18,F,[ 41.70915584 -72.585489 ],41.709156,-72.585489,41.709156,-72.585489,...,[1394.73483854 473.09881309],1394.734839,473.098813,1.0,10.811851,31.674134,353.796779,23.456943,4.614582,5.083221
16116,700,1577.5,330.0,player8,F,[ 41.70955079 -72.58546489],41.709551,-72.585465,41.709551,-72.585465,...,[850.53844855 312.2573834 ],850.538449,312.257383,1.0,15.090207,45.642655,275.629651,2.190543,0.479100,4.572207
16125,700,3644.0,535.0,player1,F,[ 41.7089957 -72.58555198],41.708996,-72.585552,41.708996,-72.585552,...,[1554.53727385 575.90554082],1554.537274,575.905541,1.0,4.756928,58.363710,34.356824,3.248151,0.693927,4.680822


In [45]:
distance_df.replace([np.inf], 20, inplace=True)
distance_df.replace([-np.inf], -20, inplace=True)
distance_df.loc[(~distance_df["pass_distance"].isna()) & (~distance_df["nbr_of_interceptors"].isna()) & (~distance_df["nearest_defender_dist"].isna()) & (~distance_df["nearest_defender_angle"].isna()) & (~distance_df["nearest_defender_speed"].isna()) & (~distance_df["nearest_defender_speed_rel"].isna()) & (~distance_df["receiver_speed"].isna()), 'predictions_proba'] = logmodel.predict(distance_df[(~distance_df["pass_distance"].isna()) & (~distance_df["nbr_of_interceptors"].isna()) & (~distance_df["nearest_defender_dist"].isna()) & (~distance_df["nearest_defender_angle"].isna()) & (~distance_df["nearest_defender_speed"].isna()) & (~distance_df["nearest_defender_speed_rel"].isna()) & (~distance_df["receiver_speed"].isna())][["pass_distance", "nbr_of_interceptors", "nearest_defender_dist", "nearest_defender_angle", "nearest_defender_speed", "nearest_defender_speed_rel", "receiver_speed"]])

In [46]:
distance_df.loc[250]

frame                                                    13
x_mid                                                3722.0
y_mid                                                 574.0
Name                                               player12
Team                                                      F
lon_lat                         [ 41.70898046 -72.58557925]
lon                                                41.70898
lat                                              -72.585579
lon_prev                                          41.708978
lat_prev                                         -72.585579
haversine                                          0.000062
haversine_smooth                                        NaN
mph                                                6.477642
mph_smoothed                                            NaN
haversine_smooth_10                                0.000078
mph_smoothed_10                                    8.151098
ft_to_ball                              

In [47]:
distance_df.loc[(~distance_df["pass_distance"].isna()) & (~distance_df["nbr_of_interceptors"].isna()) & (~distance_df["nearest_defender_dist"].isna()) & (~distance_df["nearest_defender_angle"].isna()) & (~distance_df["nearest_defender_speed"].isna()) & (~distance_df["nearest_defender_speed_rel"].isna()) & (~distance_df["receiver_speed"].isna())]

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,x_out,y_out,nbr_of_interceptors,nearest_defender_dist,pass_distance,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed,predictions_proba
251,13,1971.5,945.5,player6,F,[ 41.70936765 -72.58578568],41.709368,-72.585786,41.709366,-72.585787,...,1040.767709,786.114169,1.0,12.185951,86.359569,320.285265,8.565371,0.757999,11.299970,0.956358
253,13,1532.5,385.5,player8,F,[ 41.70954397 -72.58550959],41.709544,-72.585510,41.709544,-72.585510,...,845.199344,371.806960,2.0,16.709862,66.566258,291.556501,1.007441,0.314343,3.204904,0.920272
257,13,2259.0,337.0,player10,F,[ 41.70938506 -72.58544049],41.709385,-72.585440,41.709385,-72.585441,...,1116.273885,348.192604,0.0,44.843106,67.186189,236.893959,1.007441,0.191975,5.247773,0.990734
263,13,3448.5,706.0,player1,F,[ 41.70907653 -72.58567 ],41.709077,-72.585670,41.709076,-72.585671,...,1452.893055,687.001205,0.0,36.614840,34.727944,206.175723,2.174890,0.353925,6.145062,0.959567
270,13,936.5,514.5,player9,F,[ 41.70961894 -72.5856072 ],41.709619,-72.585607,41.709620,-72.585607,...,663.381461,475.616014,2.0,3.322918,70.452329,144.044071,1.630989,0.338714,4.815245,0.911276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45625,1961,1746.5,1367.5,player4,H,[ 41.7093651 -72.5859003],41.709365,-72.585900,41.709365,-72.585901,...,1007.152069,949.920317,3.0,6.992620,68.514787,318.741738,1.309285,0.393566,3.326721,0.839091
45626,1961,85.0,506.5,player17,H,[ 41.70975663 -72.58561857],41.709757,-72.585619,41.709757,-72.585619,...,353.066075,439.333909,4.0,19.131279,88.497602,60.629898,0.924594,1.383842,0.668136,0.891780
45629,1961,1713.0,773.0,player22,H,[ 41.70943053 -72.58572131],41.709431,-72.585721,41.709431,-72.585721,...,960.258830,686.912607,1.0,16.601501,41.905738,178.296508,2.534570,2.067163,1.226110,0.957808
45631,1961,814.5,559.0,player21,H,[ 41.70962623 -72.58563296],41.709626,-72.585633,41.709627,-72.585632,...,636.144099,510.159138,3.0,23.568907,73.557666,89.730583,2.748246,1.008884,2.724046,0.834415


In [48]:
distance_df[~distance_df['predictions_proba'].isna()]

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,x_out,y_out,nbr_of_interceptors,nearest_defender_dist,pass_distance,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed,predictions_proba
251,13,1971.5,945.5,player6,F,[ 41.70936765 -72.58578568],41.709368,-72.585786,41.709366,-72.585787,...,1040.767709,786.114169,1.0,12.185951,86.359569,320.285265,8.565371,0.757999,11.299970,0.956358
253,13,1532.5,385.5,player8,F,[ 41.70954397 -72.58550959],41.709544,-72.585510,41.709544,-72.585510,...,845.199344,371.806960,2.0,16.709862,66.566258,291.556501,1.007441,0.314343,3.204904,0.920272
257,13,2259.0,337.0,player10,F,[ 41.70938506 -72.58544049],41.709385,-72.585440,41.709385,-72.585441,...,1116.273885,348.192604,0.0,44.843106,67.186189,236.893959,1.007441,0.191975,5.247773,0.990734
263,13,3448.5,706.0,player1,F,[ 41.70907653 -72.58567 ],41.709077,-72.585670,41.709076,-72.585671,...,1452.893055,687.001205,0.0,36.614840,34.727944,206.175723,2.174890,0.353925,6.145062,0.959567
270,13,936.5,514.5,player9,F,[ 41.70961894 -72.5856072 ],41.709619,-72.585607,41.709620,-72.585607,...,663.381461,475.616014,2.0,3.322918,70.452329,144.044071,1.630989,0.338714,4.815245,0.911276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45625,1961,1746.5,1367.5,player4,H,[ 41.7093651 -72.5859003],41.709365,-72.585900,41.709365,-72.585901,...,1007.152069,949.920317,3.0,6.992620,68.514787,318.741738,1.309285,0.393566,3.326721,0.839091
45626,1961,85.0,506.5,player17,H,[ 41.70975663 -72.58561857],41.709757,-72.585619,41.709757,-72.585619,...,353.066075,439.333909,4.0,19.131279,88.497602,60.629898,0.924594,1.383842,0.668136,0.891780
45629,1961,1713.0,773.0,player22,H,[ 41.70943053 -72.58572131],41.709431,-72.585721,41.709431,-72.585721,...,960.258830,686.912607,1.0,16.601501,41.905738,178.296508,2.534570,2.067163,1.226110,0.957808
45631,1961,814.5,559.0,player21,H,[ 41.70962623 -72.58563296],41.709626,-72.585633,41.709627,-72.585632,...,636.144099,510.159138,3.0,23.568907,73.557666,89.730583,2.748246,1.008884,2.724046,0.834415


In [49]:
distance_df.rename(columns={"predictions_proba":"Pass_Prob"}, inplace=True)

## Join Back to Video Data

In [50]:
import math

out_fps = 29.3

run = 16
path = f"exp{run}/labels/Hurling_intersquad_passes.txt"

file_path = "".join(path.split(".")[:-1]).replace("labels/", "") + ".mov"  # change to your own video path
# print(file_path)
vid = cv2.VideoCapture(file_path)
vid_height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
vid_width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
# print(vid_height, vid_width)

img = cv2.imread('pitch.png')
height,width,layers=img.shape
print(width, height, layers)

ratio = width / vid_width, height / vid_height
print(ratio)

video=cv2.VideoWriter("".join(path.split(".")[:-1]).replace("labels/", "") + "_analysis_predict_pass.mp4" ,cv2.VideoWriter_fourcc(*'mp4v'),out_fps,(width,height))
# print(video)

for j in distance_df.frame.unique():
    img = cv2.imread('pitch.png')
    image = cv2.putText(img, str(j), (100,100), fontFace=2, fontScale=.8, color=(0, 0, 0), thickness=2, lineType = cv2.LINE_AA)
    # print(img)
    frame = distance_df[distance_df['frame'] == j].to_dict(orient='records')
    for position in frame:
        # print(position)
        #image = cv2.circle(img, (j*5,100), radius=10, color=(255, 0, 0), thickness=-1)
        if set(["Name", "x_out", "y_out", "Team"]) <= set(position.keys()):
            if position["Team"] == "F":
                color = (255, 0, 0)
            elif position["Team"] == "H":
                color = (0, 0, 255)
            elif position['Name'] == "ball":
                color = (0, 0, 0)
            else:
                color = (0, 0, 0)
            position['Name'] = position['Name'] if position['Name'] != "ball" else "*"
            if (not math.isnan(position['x_out'])) and (not math.isnan(position['y_out'])):
                if math.isnan(position['mph_smoothed']):
                    position['mph_smoothed'] = 0
                if math.isnan(position['ft_to_ball']):
                    position['ft_to_ball'] = 0
                # print("Writing Vid")
                image = cv2.putText(img, "."+position['Name'], (int(position['x_out']), int(position['y_out'])), fontFace=2, fontScale=.8, color=color, thickness=2, lineType = cv2.LINE_AA)
                if not math.isnan(position['Pass_Prob']):
                    image = cv2.putText(img, "Pass_Prob: " + str(round(position['Pass_Prob'] * 100)) + "%", (int(position['x_out']), int(position['y_out'] + 20)), fontFace=2, fontScale=.5, color=color, thickness=1, lineType = cv2.LINE_AA)
                # cv2.imshow("Output", image)
                # if cv2.waitKey(1) & 0xFF == ord('q'):
                #     break

    video.write(img)

cv2.destroyAllWindows()
video.release()

1998 1412 3
(0.5203125, 0.6537037037037037)


In [51]:
position

{'frame': 1961,
 'x_mid': 1070.0,
 'y_mid': 453.0,
 'Name': 'player10',
 'Team': 'F',
 'lon_lat': '[ 41.70961417 -72.58556846]',
 'lon': 41.70961416659841,
 'lat': -72.58556845522509,
 'lon_prev': 41.70961377605306,
 'lat_prev': -72.58556807314984,
 'haversine': 2.7606526256650663e-05,
 'haversine_smooth': 2.874869753270667e-05,
 'mph': 2.882121341194329,
 'mph_smoothed': 3.001364022414577,
 'haversine_smooth_10': 2.632419692005854e-05,
 'mph_smoothed_10': 2.748246158454112,
 'ft_to_ball': 87.17935769020383,
 'ft_traveled': 509.8394485326005,
 'pixels_out': '[691.00995199 422.98097667]',
 'x_out': 691.0099519949445,
 'y_out': 422.9809766735938,
 'nbr_of_interceptors': nan,
 'nearest_defender_dist': nan,
 'pass_distance': nan,
 'nearest_defender_angle': nan,
 'nearest_defender_speed': nan,
 'nearest_defender_speed_rel': nan,
 'receiver_speed': nan,
 'Pass_Prob': nan}

In [52]:
distance_df

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,x_out,y_out,nbr_of_interceptors,nearest_defender_dist,pass_distance,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed,Pass_Prob
0,3,1510.0,449.5,Eric,H,[ 41.70953108 -72.58555477],41.709531,-72.585555,NaN,NaN,...,850.208669,435.280573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,3806.5,583.5,ball,N,[ 41.70895713 -72.58558384],41.708957,-72.585584,NaN,NaN,...,1588.245004,615.799023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1542.5,390.5,player8,F,[ 41.70954044 -72.58551292],41.709540,-72.585513,NaN,NaN,...,850.051490,377.454043,2.0,15.302462,68.741884,289.537910,NaN,NaN,NaN,NaN
7,3,880.5,494.5,player3,H,[ 41.70963465 -72.58559718],41.709635,-72.585597,NaN,NaN,...,636.545624,455.615082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,3373.5,919.5,player7,H,[ 41.70911253 -72.58577236],41.709113,-72.585772,NaN,NaN,...,1393.888124,800.413960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45632,1961,533.5,430.0,player13,F,[ 41.70971542 -72.5855676 ],41.709715,-72.585568,41.709716,-72.585568,...,479.947029,380.028737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45633,1961,1977.5,553.0,Referee,R,[ 41.70941444 -72.58560814],41.709414,-72.585608,41.709415,-72.585608,...,1023.726387,540.987959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45634,1961,3149.0,801.0,player7,H,[ 41.70915327 -72.58572214],41.709153,-72.585722,41.709153,-72.585721,...,1354.453183,735.260403,1.0,1.442488,12.569617,12.989707,8.776878,0.881311,9.958884,0.335373
45635,1961,3207.5,800.0,player1,F,[ 41.70914046 -72.58572117],41.709140,-72.585721,41.709143,-72.585721,...,1370.134168,735.968718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
pd.DataFrame(logmodel.predict_proba(X=distance_df.loc[distance_df.index==45634][["pass_distance", "nbr_of_interceptors", "nearest_defender_dist", "nearest_defender_angle", "nearest_defender_speed", "nearest_defender_speed_rel", "receiver_speed"]]))[1]

AttributeError: 'LogitResults' object has no attribute 'predict_proba'

In [ ]:
distance_df.loc[(~distance_df["pass_distance"].isna()) & (~distance_df["nbr_of_interceptors"].isna()) & (~distance_df["nearest_defender_dist"].isna()) & (~distance_df["nearest_defender_angle"].isna()) & (~distance_df["nearest_defender_speed"].isna()) & (~distance_df["nearest_defender_speed_rel"].isna()) & (~distance_df["receiver_speed"].isna())]

,frame,x_mid,y_mid,Name,Team,lon_lat,lon,lat,lon_prev,lat_prev,...,x_out,y_out,nbr_of_interceptors,nearest_defender_dist,pass_distance,nearest_defender_angle,nearest_defender_speed,nearest_defender_speed_rel,receiver_speed,Pass_Prob
250,13,1971.5,945.5,Emmet,F,[ 41.70936765 -72.58578568],41.709368,-72.585786,41.709366,-72.585787,...,1040.767709,786.114169,1.0,12.185951,86.359569,320.285265,8.565371,0.757999,11.299970,0.967169
257,13,3448.5,706.0,James,F,[ 41.70907653 -72.58567 ],41.709077,-72.585670,41.709076,-72.585671,...,1452.893055,687.001205,0.0,36.614840,34.727944,206.175723,2.174890,0.353925,6.145062,0.931166
265,13,1532.5,385.5,Droney,F,[ 41.70954397 -72.58550959],41.709544,-72.585510,41.709544,-72.585510,...,845.199344,371.806960,2.0,16.709862,66.566258,291.556501,1.007441,0.314343,3.204904,0.791413
266,13,936.5,514.5,Muirrean,F,[ 41.70961894 -72.5856072 ],41.709619,-72.585607,41.709620,-72.585607,...,663.381461,475.616014,2.0,3.322918,70.452329,144.044071,1.630989,0.338714,4.815245,0.512291
270,13,2259.0,337.0,Liam,F,[ 41.70938506 -72.58544049],41.709385,-72.585440,41.709385,-72.585441,...,1116.273885,348.192604,0.0,44.843106,67.186189,236.893959,1.007441,0.191975,5.247773,0.955023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45626,1961,1746.5,1367.5,Ian,H,[ 41.7093651 -72.5859003],41.709365,-72.585900,41.709365,-72.585901,...,1007.152069,949.920317,3.0,6.992620,68.514787,318.741738,1.309285,0.393566,3.326721,NaN
45627,1961,1713.0,773.0,Sean,H,[ 41.70943053 -72.58572131],41.709431,-72.585721,41.709431,-72.585721,...,960.258830,686.912607,1.0,16.601501,41.905738,178.296508,2.534570,2.067163,1.226110,NaN
45628,1961,391.5,594.0,Eric,H,[ 41.70968157 -72.58565621],41.709682,-72.585656,41.709682,-72.585656,...,505.560604,525.994443,2.0,22.792548,75.033318,339.652452,2.534570,2.738313,0.925595,NaN
45631,1961,483.5,764.0,Dylan,H,[ 41.70962146 -72.58572642],41.709621,-72.585726,41.709622,-72.585726,...,598.753854,651.716621,1.0,7.485309,62.456308,21.364700,2.534570,1.563086,1.621516,NaN
